<a href="https://colab.research.google.com/github/Ammad12345390/RAG_Chatbot-/blob/main/RAG_rag_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -U langchain-community langchain-openai faiss-cpu pypdf python-docx docx2txt openai

In [12]:
import os
from getpass import getpass
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI

In [13]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


embedding = OpenAIEmbeddings()


FOLDER_PATH = "./all_filesz"
INDEX_PATH = "./faiss_index"

Enter your OpenAI API key: ··········


In [14]:
import glob
import os

departments = ["marketing", "policies", "technical_sops", "compliance_and_legal"]

all_files = {}

for dept in departments:
    folder_path = f"/content/all_filesz/{dept}/*.docx"
    files = glob.glob(folder_path)
    # Store with department name
    all_files[dept] = files

# Example: print each file with its department
for dept, files in all_files.items():
    for f in files:
        print(f"{os.path.basename(f)} --> {dept}")


Brand_Guidelines_Handbook.docx --> marketing
Q3_Content_Marketing_Strategy.docx --> marketing
Product_Launch_Campaign_Report.docx --> marketing
Employee_Leave_&_Absence_Policy.docx --> policies
Workplace_Code_of_Conduct.docx --> policies
Remote_Work_&_Hybrid_Policy.docx --> policies
Backend_Service_Monitoring_SOP.docx --> technical_sops
Production_Incident_Response_Runbook.docx --> technical_sops
CI-CD_Deployment_Pipeline_Guide.docx --> technical_sops
Contract_Review_Checklist.docx --> compliance_and_legal
GDPR_Compliance_Guidelines.docx --> compliance_and_legal
HIPAA_Data_Handling_Policy.docx --> compliance_and_legal


In [15]:
all_files

{'marketing': ['/content/all_filesz/marketing/Brand_Guidelines_Handbook.docx',
  '/content/all_filesz/marketing/Q3_Content_Marketing_Strategy.docx',
  '/content/all_filesz/marketing/Product_Launch_Campaign_Report.docx'],
 'policies': ['/content/all_filesz/policies/Employee_Leave_&_Absence_Policy.docx',
  '/content/all_filesz/policies/Workplace_Code_of_Conduct.docx',
  '/content/all_filesz/policies/Remote_Work_&_Hybrid_Policy.docx'],
 'technical_sops': ['/content/all_filesz/technical_sops/Backend_Service_Monitoring_SOP.docx',
  '/content/all_filesz/technical_sops/Production_Incident_Response_Runbook.docx',
  '/content/all_filesz/technical_sops/CI-CD_Deployment_Pipeline_Guide.docx'],
 'compliance_and_legal': ['/content/all_filesz/compliance_and_legal/Contract_Review_Checklist.docx',
  '/content/all_filesz/compliance_and_legal/GDPR_Compliance_Guidelines.docx',
  '/content/all_filesz/compliance_and_legal/HIPAA_Data_Handling_Policy.docx']}

In [16]:
FOLDER_PATH = "/content/all_filesz"
INDEX_PATH = "/content/faiss_index"

# Define department folders
departments = ["marketing", "policies", "technical_sops", "compliance_and_legal"]

embedding = OpenAIEmbeddings()

# ✅ Load existing FAISS index if available
if os.path.exists(f"{INDEX_PATH}/index.faiss"):
    print("✅ Loading existing FAISS index...")
    vectorstore = FAISS.load_local(INDEX_PATH, embedding, allow_dangerous_deserialization=True)

else:
    if not os.path.exists(FOLDER_PATH):
        os.makedirs(FOLDER_PATH)
        print(f"📂 Created '{FOLDER_PATH}'. Please add department subfolders and rerun.")
        raise SystemExit

    all_documents = []

    # 🔎 Loop through each department folder
    for dept in departments:
        folder_path = f"{FOLDER_PATH}/{dept}/*"
        files = glob.glob(folder_path)

        for file in files:
            file_name, ext = os.path.splitext(os.path.basename(file))
            file_name = file_name.strip()

            loader = None
            if ext.lower() == ".pdf":
                loader = PyPDFLoader(file)
            elif ext.lower() == ".docx":
                loader = Docx2txtLoader(file)
            else:
                continue  # skip unsupported formats

            try:
                documents = loader.load()

                for i, doc in enumerate(documents, 1):
                    doc.metadata.update({
                        "source_file": os.path.basename(file),
                        "chunk_number": i,
                        "page_number": i,
                        "department": dept,     # ✅ taken from folder name
                        "title": file_name      # ✅ simple file name as title
                    })

                    # ✅ Print debug info
                    print(f"📂 Department: {dept}")
                    print(f"📄 Title: {file_name}")
                    print(f"➡️ Source: {os.path.basename(file)}, Chunk: {i}\n")

                all_documents.extend(documents)

            except Exception as e:
                print(f"⚠️ Could not load {file}, skipping... ({e})")
                continue

    # ⚡ Create FAISS index from documents
    vectorstore = FAISS.from_documents(all_documents, embedding)
    vectorstore.save_local(INDEX_PATH)
    print("✅ New FAISS index created and saved!")

📂 Department: marketing
📄 Title: Brand_Guidelines_Handbook
➡️ Source: Brand_Guidelines_Handbook.docx, Chunk: 1

📂 Department: marketing
📄 Title: Q3_Content_Marketing_Strategy
➡️ Source: Q3_Content_Marketing_Strategy.docx, Chunk: 1

📂 Department: marketing
📄 Title: Product_Launch_Campaign_Report
➡️ Source: Product_Launch_Campaign_Report.docx, Chunk: 1

📂 Department: policies
📄 Title: Employee_Leave_&_Absence_Policy
➡️ Source: Employee_Leave_&_Absence_Policy.docx, Chunk: 1

📂 Department: policies
📄 Title: Workplace_Code_of_Conduct
➡️ Source: Workplace_Code_of_Conduct.docx, Chunk: 1

📂 Department: policies
📄 Title: Remote_Work_&_Hybrid_Policy
➡️ Source: Remote_Work_&_Hybrid_Policy.docx, Chunk: 1

📂 Department: technical_sops
📄 Title: Backend_Service_Monitoring_SOP
➡️ Source: Backend_Service_Monitoring_SOP.docx, Chunk: 1

📂 Department: technical_sops
📄 Title: Production_Incident_Response_Runbook
➡️ Source: Production_Incident_Response_Runbook.docx, Chunk: 1

📂 Department: technical_sops
📄 

In [17]:
if not all_documents:
    print(f"⚠️ No valid PDF/DOCX files found in '{FOLDER_PATH}'.")
    raise SystemExit

os.makedirs(INDEX_PATH, exist_ok=True)
vectorstore = FAISS.from_documents(all_documents, embedding)
vectorstore.save_local(INDEX_PATH)
print("✅ FAISS index created and saved with department + title metadata!")


✅ FAISS index created and saved with department + title metadata!


In [18]:
vectorstore = FAISS.load_local(INDEX_PATH, embedding, allow_dangerous_deserialization=True)

# Get the total number of documents
total_docs = len(vectorstore.docstore._dict)  # FAISS stores docs in docstore
print(f"Total documents in vectorstore: {total_docs}")

Total documents in vectorstore: 12


In [19]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(all_documents)


vectorstore = FAISS.from_documents(chunks, embedding)
vectorstore.save_local(INDEX_PATH)
print("✅ FAISS index created and saved!")



✅ FAISS index created and saved!


In [20]:
# Create retriever with filters inside
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 2,  # number of docs
        "filter": {
            "$or": [
                {"department": "marketing"},
                {"department": "policies"},
                {"department": "technical_sops"},
                {"department": "compliance_and_legal"}
            ]
        }
    }
)

# Now just call retriever without repeating filters
results = retriever.get_relevant_documents("GDPR rules")

# Print nicely
for doc in results:
    print(f"📂 Department: {doc.metadata.get('department')}")
    print(f"📄 Title: {doc.metadata.get('title')}")
    print(f"Content: {doc.page_content[:60]}...\n")


/tmp/ipython-input-3044039518.py:17: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents("GDPR rules")


📂 Department: compliance_and_legal
📄 Title: GDPR_Compliance_Guidelines
Content: GDPR Compliance Guidelines

Department: Legal

Jurisdiction:...

📂 Department: compliance_and_legal
📄 Title: GDPR_Compliance_Guidelines
Content: Page 2

User Consent Requirements
- Consent must be freely g...



In [21]:
policy_prompt = """
You are a Company Policy Assistant. Only answer questions using the information provided in the
company's policy documents.
If the answer is not found in the policy,
say: I cannot find that information in the policy.
 Be concise, professional, and accurate."""

In [25]:
print("Chatbot ready! Type 'exit' to stop.")

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    k = 1
    relevant_docs = retriever.get_relevant_documents(query)[:k]

    # Combine content and metadata for context
    context_list = []
    for i, doc in enumerate(relevant_docs, 1):
        context_list.append(
            f"Chunk {i} Content:\n{doc.page_content}\nMetadata: {doc.metadata}\n"
        )
    context = "\n".join(context_list)


    print("\n=== Retrieved Chunks ===")
    for doc in relevant_docs:
        print("Content:", doc.page_content[:40], "...")
        print("Metadata:", doc.metadata)
        print("-" * 30)

    messages = [
        {"role": "system", "content": policy_prompt},
        {"role": "user", "content": f"Context from documents:\n{context}\n\nQuestion: {query}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=messages,
        max_tokens=300,
        temperature=0.7
    )

    print("Bot:", response.choices[0].message.content)

Chatbot ready! Type 'exit' to stop.


KeyboardInterrupt: Interrupted by user